# Sprawozdanie 6

## Mixed-integer linear programing

In [1]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
from pathlib import Path
from ortools.sat.python import cp_model


class RPQ():
    def __init__(self, r, p, q):
        self.R = r
        self.P = p
        self.Q = q


def Milp(jobs, instanceName):
    variableMaxValue = sum(job.R+job.P+job.Q for job in jobs)

    solver = pywraplp.Solver('simple_mip_program',
            pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    alfasMatrix = {}
    for i in range(len(jobs)):
        for j in range(len(jobs)):
            alfasMatrix[i, j] = solver.IntVar(0, 1, "alfa"+str(i) + "_" + str(j))

    starts = [(solver.IntVar(0, variableMaxValue, "starts"+str(i))) for i in range(len(jobs))]
    cmax = solver.IntVar(0, variableMaxValue, "cmax")

    for job, start in zip(jobs, starts):
        solver.Add(start >= job.R)
        solver.Add(cmax >= start + job.P + job.Q)

    for i in range(len(jobs)):
        for j in range(i+1, len(jobs)):
            solver.Add(starts[i]+jobs[i].P <= starts[j] + alfasMatrix [i,j]*variableMaxValue)
            solver.Add(starts[j]+jobs[j].P <= starts[i] + alfasMatrix[j,i] *variableMaxValue)
            solver.Add(alfasMatrix[i,j] + alfasMatrix[j,i] == 1)

    solver.Minimize(cmax)
    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        print("Not optimal!")
    print(instanceName, "Cmax:", solver.Objective().Value())
    pi = [(i, starts[i].solution_value()) for i in range(len(starts))]

    pi.sort(key=lambda x: x[1])
    print(pi)


def GetRPQsFromFile (file_path):

    full_file = Path(file_path).read_text()
    words = full_file.replace("\n", " ").split(" ")
    words_cleaned = list(filter(None, words))
    numbers = list(map(int, words_cleaned))

    jobs = []
    for i in range(numbers.pop(0)):
        jobs.append(RPQ(numbers[0], numbers[1], numbers[2]))
        numbers.pop(0)
        numbers.pop(0)
        numbers.pop(0)
    return jobs


files = ["./dane rpq/data000.txt"]
for file in files:
    jobs = GetRPQsFromFile(file)
    Milp(jobs, file)


./dane rpq/data000.txt Cmax: 228.0
[(0, 0.0), (2, 27.0), (1, 140.0), (3, 147.0)]


## Constrain Programing


In [2]:
from or_tools_milp_cp import GetRPQsFromFile
from ortools.sat.python import cp_model
from job import Job
from ortools.linear_solver import pywraplp


def cp(jobs, instanceName):
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()

    # variableMaxValue = sum(job.R+job.P+job.Q for job in jobs)
    # r = model.NewIntVar(0, variableMaxValue, 'r')
    # p = model.NewIntVar(0, variableMaxValue, 'p')
    # q = model.NewIntVar(0, variableMaxValue, 'q')
    # model.Add()
    variableMaxValue = sum(job.R+job.P+job.Q for job in jobs)

    alfasMatrix = {}
    for i in range(len(jobs)):
        for j in range(len(jobs)):
            alfasMatrix[i, j] = model.NewIntVar(0, 1, "alfa"+str(i) + "_" + str(j))

    starts = [(model.NewIntVar(0, variableMaxValue, "starts"+str(i))) for i in range(len(jobs))]
    cmax = model.NewIntVar(0, variableMaxValue, "cmax")

    for job, start in zip(jobs, starts):
        model.Add(start >= job.R)
        model.Add(cmax >= start + job.P + job.Q)

    for i in range(len(jobs)):
        for j in range(i+1, len(jobs)):
            model.Add(starts[i]+jobs[i].P <= starts[j] + alfasMatrix [i,j]*variableMaxValue)
            model.Add(starts[j]+jobs[j].P <= starts[i] + alfasMatrix[j,i] *variableMaxValue)
            model.Add(alfasMatrix[i,j] + alfasMatrix[j,i] == 1)

    model.Minimize(cmax)
    status = solver.Solve(model)

    print(solver.ObjectiveValue())

file = "./dane rpq/data000.txt"
jobs = GetRPQsFromFile(file)
cp(jobs, file)

228.0


## CP dla problemu PD

In [8]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
from pathlib import Path
from ortools.sat.python import cp_model
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import solver_parameters_pb2


class PWD():
    def __init__(self, p, w, d):
        self.P = p
        self.W = w
        self.D = d


def CP_WT(jobs, instanceName):
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    sumaP = [job.P for job in jobs]
    variableMaxValue = sum(job.W*(sum(sumaP)-job.D) if sum(sumaP)-job.D > 0 else 0 for job in jobs)

    starts = [(model.NewIntVar(0, variableMaxValue, "starts"+str(i))) for i in range(len(jobs))]
    penalty = [(model.NewIntVar(0, variableMaxValue, "kara" + str(i))) for i in range(len(jobs))]

    alfasMatrix = {}
    for i in range(len(jobs)):
        for j in range(len(jobs)):
            alfasMatrix[i, j] = model.NewIntVar(0, 1, "alfa"+str(i) + "_" + str(j))

    for i in range(len(jobs)):
        for j in range(i+1, len(jobs)):
            model.Add(starts[i]+jobs[i].P <= starts[j] + alfasMatrix [i,j]*variableMaxValue)
            model.Add(starts[j]+jobs[j].P <= starts[i] + alfasMatrix[j,i] *variableMaxValue)
            model.Add(alfasMatrix[i,j] + alfasMatrix[j,i] == 1)
        model.Add(penalty[i] >= (starts[i]+jobs[i].P-jobs[i].D)*jobs[i].W)

    model.Minimize(sum(penalty))
    status = solver.Solve(model)
    pi = [(i, start.GetVarValueMap()) for start in starts]
    print(instanceName, "Suma CP:", solver.ObjectiveValue())
    return solver.ObjectiveValue()
    

def GetPWDsFromFile (file_path):

    full_file = Path(file_path).read_text()
    words = full_file.replace("\n", " ").split(" ")
    words_cleaned = list(filter(None, words))
    numbers = list(map(int, words_cleaned))

    jobs = []
    for i in range(numbers.pop(0)):
        jobs.append(PWD(numbers[0], numbers[1], numbers[2]))
        numbers.pop(0)
        numbers.pop(0)
        numbers.pop(0)
    return jobs

res = []
exp = [799, 742, 688]
files = ["./dane pwd/data11.txt", "./dane pwd/data12.txt", "./dane pwd/data13.txt"]
for file in files:
    jobs = GetPWDsFromFile(file)
    res.append(CP_WT(jobs, file))

for rslt, dt  in zip(res, exp):
    print(" uzyskano: "+str(rslt)+" oczekiwane: "+str(dt))

./dane pwd/data11.txt Suma CP: 799.0
./dane pwd/data12.txt Suma CP: 742.0
./dane pwd/data13.txt Suma CP: 688.0
 uzyskano: 799.0 oczekiwane: 799
 uzyskano: 742.0 oczekiwane: 742
 uzyskano: 688.0 oczekiwane: 688


## CP dla problemu przepływowego


In [3]:
from ortools.sat.python import cp_model
from job import Job


def load_from_file(file_name):
    joblist = []
    with open(file_name) as file:
        line = file.readline()
        line = file.readline()
        while "data" not in line:
            pass
            line = file.readline()
        line = file.readline()
        line = list(map(int, line.split()))

        for i, line in enumerate(file.readlines()):
            if line != '\n':
                joblist.append(Job(list(map(int, line.split())), index=i, name="zadanie " + str(i + 1)))
    return joblist


def cp_js(jobs):
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()

    variableMaxValue = sum(job.time[i] for i in range(len(jobs[0].time)) for job in jobs )

    alfasMatrix = {}
    for i in range(len(jobs)):
        for j in range(len(jobs)):
            alfasMatrix[i, j] = model.NewIntVar(0, 1, "alfa"+str(i) + "_" + str(j))

    starts = [[(model.NewIntVar(0, variableMaxValue, "starts"+str(i)+"machine"+str(j))) for i in range(len(jobs[0].time))] for j in range(len(jobs))]
    cmax = model.NewIntVar(0, variableMaxValue, "cmax")

    for i in range(len(jobs)):
        for j in range(i+1, len(jobs)):
            for k in range(len(jobs[0].time)):
                model.Add(starts[i][k]+jobs[i].time[k] <= starts[j][k] + alfasMatrix [i,j]*variableMaxValue)
                model.Add(starts[j][k]+jobs[j].time[k] <= starts[i][k] + alfasMatrix[j,i] *variableMaxValue)
                model.Add(alfasMatrix[i,j] + alfasMatrix[j,i] == 1)
        model.Add(cmax >= starts[i][k] + jobs[i].time[k])

    model.Minimize(cmax)
    status = solver.Solve(model)
    pi = [(i, starts[i][0].GetVarValueMap()) for i in range(len(starts))]
    # pi.sort(key=lambda x: x[1])
    print(pi)
    # cmax.GetVarValueMap()
    print(solver.ObjectiveValue())


jobs = load_from_file("insa/ta01")
cp_js(jobs)


[(0, (defaultdict(<class 'int'>, {starts0machine0(0..59): 1}), 0)), (1, (defaultdict(<class 'int'>, {starts0machine1(0..59): 1}), 0)), (2, (defaultdict(<class 'int'>, {starts0machine2(0..59): 1}), 0)), (3, (defaultdict(<class 'int'>, {starts0machine3(0..59): 1}), 0))]
18.0
